In [ ]:
import time
import csv
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.support.ui import WebDriverWait

In [ ]:
options=Options()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
options.add_experimental_option("detach",True)
driver= webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

In [ ]:
glink= "https://www.google.com/search?q=site:youtube.com+openinapp.co&num=100&filter=0&start="
c=0
urls=[]
prev=len(urls)
timeout=35

In [ ]:
while(prev<10000):
    driver.get(glink+str(c))

    if not (len(driver.find_elements(By.ID, "OotqVd"))==0):
        break
    
    try:
        element_present = expected_conditions.presence_of_element_located((By.CLASS_NAME, "DhN8Cf"))
        WebDriverWait(driver, timeout).until(element_present)
    except TimeoutError:
        break

    for element in driver.find_elements(By.CLASS_NAME, "DhN8Cf"):
        url= element.find_element(By.TAG_NAME, 'a').get_attribute('href')
        if 'hashtag' not in url:
            urls.append(url)
    for element in driver.find_elements(By.CLASS_NAME, "yuRUbf"):
        url= element.find_element(By.TAG_NAME, 'a').get_attribute('href')    
        if 'hashtag' not in url:
            urls.append(url)
    cur=len(urls)
    print(cur)
    if(cur>prev):
        prev=len(urls)
    else:
        break
    c+=100

print("FINAL: ", len(urls))

In [ ]:
for url in urls:
    driver.get(url)
    time.sleep(1)
    
    try:
        if 'https://www.youtube.com/c/' in url:
            link=url.replace('c/','').replace('/community','')

        elif 'https://www.youtube.com/watch?' in url:
            link= driver.find_element('xpath', "//ytd-video-owner-renderer[@class='style-scope ytd-watch-metadata']//a[@class='yt-simple-endpoint style-scope ytd-video-owner-renderer']").get_attribute("href")

        elif 'https://www.youtube.com/post/' in url:
            link= driver.find_element(By.CLASS_NAME, "style-scope ytd-backstage-post-renderer").find_element('xpath', ".//a[@class='yt-simple-endpoint style-scope ytd-backstage-post-renderer']").get_attribute("href")

        elif 'https://www.youtube.com/channel/' in url:
            url='https://www.youtube.com/channel/'+ url.split('channel/')[1].split('/')[0]
            driver.get(url)
            handle= driver.find_element(By.XPATH, "//div[@id='meta']//span//yt-formatted-string[@id='channel-handle']").get_attribute('innerHTML')
            handle=handle.replace('@','')
            link='https://www.youtube.com/'+handle
        else:
            print(link)
            outlier.append(link)
            
        channel_urls.append(link)
    except:
        continue

In [ ]:
channels=[]
for url in channel_urls:
    url=url.replace('?app=desktop', '')
    url=url.replace('@', '')
    channels.append(url.split('com/')[1])
len(channels)

In [ ]:
unique= list(set(channels))
pairs=[]
for i,name in enumerate(unique):
    pair=[]
    pair.append(name)
    pair.append('https://www.youtube.com/'+name)
    pairs.append(pair)
    
fields=['Channel Name', 'Channel Link']


with open('output.csv', 'w') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(fields)
    csvwriter.writerows(pairs)